# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [50]:
import geoviews as gv

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,21.92,77,100,5.25,ZA,1675778712
1,1,lebu,-37.6167,-73.6500,17.73,65,0,6.99,CL,1675778715
2,2,ouadda,8.0777,22.4007,37.43,7,1,3.40,CF,1675778718
3,3,waipawa,-41.4122,175.5153,13.47,83,4,2.61,NZ,1675778719
4,4,busselton,-33.6500,115.3333,18.22,83,0,6.85,AU,1675778721


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [55]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
#ideal_city_data_df = city_data_df.loc[city_data_df.Humidity <= 55]
city_data_df.drop(city_data_df[city_data_df["Humidity"] >= 55].index, inplace = True)
city_data_df.drop(city_data_df[city_data_df["Max Temp"] <= 16].index, inplace = True)
# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,ouadda,8.0777,22.4007,37.43,7,1,3.40,CF,1675778718
7,7,atakpame,7.5333,1.1333,35.60,28,97,2.96,TG,1675778725
33,33,bilma,18.6853,12.9164,28.07,11,74,1.36,NE,1675778739
35,35,sao joao da barra,20.5113,76.2129,23.30,26,40,2.26,IN,1675778739
36,36,bafoulabe,13.8065,-10.8321,36.67,8,0,1.89,ML,1675778740


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy(deep = True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_name = []
hotel_df['Hotel_name'] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_name
2,2,ouadda,8.0777,22.4007,37.43,7,1,3.40,CF,1675778718,
7,7,atakpame,7.5333,1.1333,35.60,28,97,2.96,TG,1675778725,
33,33,bilma,18.6853,12.9164,28.07,11,74,1.36,NE,1675778739,
35,35,sao joao da barra,20.5113,76.2129,23.30,26,40,2.26,IN,1675778739,
36,36,bafoulabe,13.8065,-10.8321,36.67,8,0,1.89,ML,1675778740,
...,...,...,...,...,...,...,...,...,...,...,...
513,513,ruwi,23.5996,58.5507,23.83,53,9,3.41,OM,1675778927,
518,518,adrar,20.5022,-10.0711,24.12,15,2,5.67,MR,1675778929,
521,521,xuddur,4.1213,43.8894,32.57,24,8,5.60,SO,1675778930,
523,523,bermejo,-32.8833,-68.8000,30.78,24,0,2.36,AR,1675778931,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
#categories = "accommodation.hotel"
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel_name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel_name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel_name']}")

# Display sample data
hotel_df

Starting hotel search
ouadda - nearest hotel: No hotel found
atakpame - nearest hotel: Roc Hotel
bilma - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
bafoulabe - nearest hotel: No hotel found
manikpur - nearest hotel: No hotel found
cape town - nearest hotel: Hotel Cinco Tierras
kikwit - nearest hotel: Grand Hôtel de Kikwit
bafq - nearest hotel: هتل رستوران علمدار
susangerd - nearest hotel: No hotel found
bargi - nearest hotel: Maikal Resort
cabo san lucas - nearest hotel: Aymara Hotel Fazenda
atar - nearest hotel: فندق سكليل
kitui - nearest hotel: Parkside Hotel
yulara - nearest hotel: Outback Pioneer Hotel & Lodge
ibra - nearest hotel: الدكيك
kolokani - nearest hotel: Hôtel kolokani
san patricio - nearest hotel: La Roche
caravelas - nearest hotel: Pousada dos Navegantes
cockburn town - nearest hotel: Coburn Hotel
san cristobal - nearest hotel: La Roche
tshikapa - nearest hotel: Hôtel TSHAMUNTU
gambela - nearest hotel: Grand Hotel
kindu - nearest hot

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_name
2,2,ouadda,8.0777,22.4007,37.43,7,1,3.40,CF,1675778718,No hotel found
7,7,atakpame,7.5333,1.1333,35.60,28,97,2.96,TG,1675778725,Roc Hotel
33,33,bilma,18.6853,12.9164,28.07,11,74,1.36,NE,1675778739,No hotel found
35,35,sao joao da barra,20.5113,76.2129,23.30,26,40,2.26,IN,1675778739,No hotel found
36,36,bafoulabe,13.8065,-10.8321,36.67,8,0,1.89,ML,1675778740,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
513,513,ruwi,23.5996,58.5507,23.83,53,9,3.41,OM,1675778927,Sheraton Oman
518,518,adrar,20.5022,-10.0711,24.12,15,2,5.67,MR,1675778929,No hotel found
521,521,xuddur,4.1213,43.8894,32.57,24,8,5.60,SO,1675778930,No hotel found
523,523,bermejo,-32.8833,-68.8000,30.78,24,0,2.36,AR,1675778931,Muras Aparte Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["Country", "Hotel_name"]
)




# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel_name)